# Personalized Travel Planner

# COD3

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer


# Fetching the data

In [2]:
hotel_details=pd.read_csv('Hotel_details.csv',delimiter=',')
hotel_rooms=pd.read_csv('Hotel_Room_attributes.csv',delimiter=',')
hotel_cost=pd.read_csv('hotels_RoomPrice.csv',delimiter=',')

In [3]:
hotel_details.head()

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR


In [4]:
hotel_rooms.head()

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"


# Data Cleaning and transformations

In [5]:
del hotel_details['id']
del hotel_rooms['id']
del hotel_details['zipcode']

In [6]:
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()

In [7]:
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)

In [8]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')

In [9]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription', 'hotelid',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'Source', 'url', 'curr'],
      dtype='object')

In [10]:
del hotel['hotelid']
del hotel['curr']
del hotel['Source']

In [11]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'url'],
      dtype='object')

In [12]:
room_no=[
     ('king',2),
   ('queen',2), 
    ('triple',3),
    ('master',3),
   ('family',4),
   ('murphy',2),
   ('quad',4),
   ('double-double',4),
   ('mini',2),
   ('studio',1),
    ('junior',2),
   ('apartment',4),
    ('double',2),
   ('twin',2),
   ('double-twin',4),
   ('single',1),
     ('diabled',1),
   ('accessible',1),
    ('suite',2),
    ('one',2)
   ]

In [13]:
def calc():
    guests_no=[]
    for i in range(hotel.shape[0]):
        temp=hotel['roomtype'][i].lower().split()
        flag=0
        for j in range(len(temp)):
            for k in range(len(room_no)):
                if temp[j]==room_no[k][0]:
                    guests_no.append(room_no[k][1])
                    flag=1
                    break
            if flag==1:
                break
        if flag==0:
            guests_no.append(2)
    hotel['guests_no']=guests_no

calc()

In [14]:
hotel['roomamenities']=hotel['roomamenities'].str.replace(': ;',',')

In [40]:
from math import sin, cos, sqrt, atan2, radians
import requests

# Replace with the environment variable
secret_value_0 = '4519d076b432f5'

R = 6373.0  # Earth's Radius
sw = stopwords.words('english')
lemm = WordNetLemmatizer()

def hybrid(address, city, number, features):
    features = features.lower()
    features_tokens = word_tokenize(features)
    f1_set = {w for w in features_tokens if not w in sw}
    f_set = set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))

    data = {
        'key': secret_value_0,
        'q': address,
        'format': 'json'
    }
    url = "https://nominatim.openstreetmap.org/search"  # Example URL, replace with actual
    response = requests.get(url, params=data)
    dist = []
    lat1, long1 = response.json()[0]['lat'], response.json()[0]['lon']
    lat1 = radians(float(lat1))
    long1 = radians(float(long1))

    hybridbase = hotel[hotel['guests_no'] == number]
    hybridbase['city'] = hybridbase['city'].str.lower()
    hybridbase = hybridbase[hybridbase['city'] == city.lower()]
    hybridbase.drop_duplicates(subset='hotelcode', inplace=True, keep='first')
    hybridbase = hybridbase.set_index(np.arange(hybridbase.shape[0]))

    for i in range(hybridbase.shape[0]):
        lat2 = radians(hybridbase['latitude'][i])
        long2 = radians(hybridbase['longitude'][i])
        dlon = long2 - long1
        dlat = lat2 - lat1
        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist.append(distance)

    hybridbase['distance'] = dist
    hybridbase = hybridbase.sort_values(by='distance', ascending=True)
    hybridbase = hybridbase.head(15)
    hybridbase = hybridbase.set_index(np.arange(hybridbase.shape[0]))

    coss = []
    for i in range(hybridbase.shape[0]):
        temp_tokens = word_tokenize(hybridbase['roomamenities'][i])
        temp1_set = {w for w in temp_tokens if not w in sw}
        temp_set = set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        coss.append(len(rvector))

    hybridbase['similarity'] = coss
    return hybridbase.sort_values(by='similarity', ascending=False).head(10)


In [41]:
url = "https://nominatim.openstreetmap.org/search"
hybrid("Big Ben London",'London',4,'I need a extra toilet and room should be completely air conditioned.I should have a bathrobe.')

C:\Users\91949\AppData\Local\Temp\ipykernel_32304\3038658495.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hybridbase['city'] = hybridbase['city'].str.lower()


,hotelcode,roomamenities,roomtype,ratedescription,hotelname,address,city,country,propertytype,starrating,latitude,longitude,url,guests_no,distance,similarity
4,127330,"Additional toilet,Air conditioning,Alarm clock...",Deluxe Family Suite,"Room size: 35 m²/377 ft², Non-smoking, 3 bathr...",Mornington Hotel London Victoria,25 Gloucester Street,london,United Kingdom,Hotels,4,51.490303,-0.140295,https://www.agoda.com/en-gb/mornington-hotel-l...,4,1.588691,2
0,79139,"Air conditioning,Bathrobes,Carpeting,Closet,Co...",St James Family King Room,"Room size: 35 m²/377 ft², Shower and bathtub, ...",The Trafalgar St. James,"2 Spring Gardens, Trafalgar Square",london,United Kingdom,Hotels,4,51.507397,-0.129185,https://www.agoda.com/en-gb/the-trafalgar-st-j...,4,0.810027,1
1,127425,"Air conditioning,Closet,Coffee/tea maker,Fan,F...",Quad Room,"Room size: 20 m²/215 ft², Shower, 2 single bed...",Melbourne House Hotel,79 Belgrave Road,london,United Kingdom,Hotels,3,51.490276,-0.137371,https://www.agoda.com/en-gb/melbourne-house-ho...,4,1.459757,1
2,84774,"Air conditioning,Coffee/tea maker,Daily housek...",Standard Quad Room,"Room size: 20 m²/215 ft², Balcony/terrace, Non...",Huttons Hotel,53-57 Belgrave Road,london,United Kingdom,Hotels,3,51.490608,-0.138419,https://www.agoda.com/en-gb/huttons-hotel/hote...,4,1.476652,1
3,127426,"Air conditioning,Alarm clock,Carpeting,Closet,...",Superior Family Room (3 Adults),"Room size: 17 m²/183 ft², Balcony/terrace, Non...",The Melita,35 Charlwood Street,london,United Kingdom,Hotels,3,51.490173,-0.138874,https://www.agoda.com/en-gb/the-melita/hotel/l...,4,1.533927,1
5,466492,"Air conditioning,Coffee/tea maker,Desk,Free Wi...",Quad Room,"Room size: 23 m²/248 ft², Non-smoking, Shower,...",St Georges Pimlico Hotel,115 St Georges Drive,london,United Kingdom,Hotels,3,51.488531,-0.139517,https://www.agoda.com/en-gb/st-georges-pimlico...,4,1.704253,1
7,148905,"Air conditioning,Closet,Coffee/tea maker,Daily...",Westminster Apartment,"Room size: 33 m²/355 ft², Street view, Non-smo...",Georgian House Hotel,37-39 St Georges Drive,london,United Kingdom,Hotels,4,51.490524,-0.143872,https://www.agoda.com/en-gb/georgian-house-hot...,4,1.751669,1
8,84263,"Air conditioning,Blackout curtains,Closet,Coff...",Quad,"Non-smoking, Shower, 2 single beds and 1 doubl...",The Grapevine Hotel,117 Warwick Way,london,United Kingdom,Hotels,2,51.490773,-0.144941,https://www.agoda.com/en-gb/the-grapevine-hote...,4,1.791608,1
9,1134132,"Air conditioning,Closet,Clothes rack,Coffee/te...",Family Double Room,"Room size: 15 m²/161 ft², Shower, 1 bunk bed",The Z Hotel City,23-28 Fleet Street,london,United Kingdom,Hotels,4,51.514295,-0.107946,https://www.agoda.com/en-gb/the-z-hotel-city/h...,4,1.900014,1
10,7488,"Air conditioning,Free Wi-Fi in all rooms!,In-r...",Standard Family,"Free Wi-Fi, Cancellation policy",Radisson Blu Edwardian Kenilworth - Bloomsbury,97 Great Russell Street,london,United Kingdom,Hotels,4,51.517822,-0.128124,https://www.agoda.com/en-gb/radisson-blu-edwar...,4,1.919850,1


In [20]:
columns_to_drop = ['id', 'refid', 'websitecode', 'dtcollected', 'ratedate', 'los', 'guests', 'roomtype', 'netrate', 
                   'ratedescription', 'ratetype', 'sourceurl', 'roomamenities', 'ispromo', 'closed', 'discount', 
                   'promoname', 'status_code', 'taxstatus', 'taxtype', 'taxamount', 'proxyused', 'israteperstay', 
                   'hotelblock', 'input_dtcollected']

# Drop only columns that exist in the DataFrame
columns_in_df = hotel_cost.columns.intersection(columns_to_drop)
hotel_cost = hotel_cost.drop(columns_in_df, axis=1)

# Display the result
hotel_cost.head()


,hotelcode,onsiterate,currency,maxoccupancy,mealinclusiontype
0,634876,82.36,USD,1,Free Breakfast
1,8328096,636.09,USD,4,NaN
2,8323442,591.74,USD,4,NaN
3,7975,881.48,USD,1,NaN
4,7975,897.53,USD,3,Free breakfast for {3}


In [22]:
hotel_cost.columns

Index(['hotelcode', 'onsiterate', 'currency', 'maxoccupancy',
       'mealinclusiontype'],
      dtype='object')

In [21]:
#To reccomend we are gonna check how much does the price vary from room to room if 
#the varience is small enough then it is better for them to recommend the hotel
hot=hotel_cost.groupby(['hotelcode','maxoccupancy'])

In [22]:
hotel_cost.sort_values(by=['onsiterate'],ascending=False)
hotel_cost=hotel_cost.drop_duplicates(subset=['hotelcode','maxoccupancy'],keep='first')

In [23]:
var = hot['onsiterate'].var().to_frame('varience')
l = []

for i in range(hotel_cost.shape[0]):
    # Access the first level of the index
    var1 = var[var.index.get_level_values(0) == hotel_cost.iloc[i, 0]]
    
    # Check if var1 is not empty
    if not var1.empty:
        # Access the second level of the index
        var2 = var1[var1.index.get_level_values(1) == hotel_cost.iloc[i, 3]]
        
        # Ensure var2 is not empty and has the expected data
        if not var2.empty:
            # Append the variance value
            l.append(var2['varience'].iloc[0])
        else:
            l.append(np.nan)  # Or another default value
    else:
        l.append(np.nan)  # Or another default value


In [24]:
hotel_cost['var']=l
hotel_cost=hotel_cost.fillna(0)
hotel_cost['mealinclusiontype']=hotel_cost['mealinclusiontype'].replace(0,'No Complimentary')

C:\Users\91949\AppData\Local\Temp\ipykernel_32304\4251150705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_cost['var']=l


In [25]:
hotel1=pd.merge(hotel,hotel_cost,left_on=['hotelcode','guests_no'],right_on=['hotelcode','maxoccupancy'],how='inner')

In [26]:
hotel1=hotel1.drop_duplicates(subset=['hotelcode','maxoccupancy'],keep='first')

In [27]:
hotel1.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'url', 'guests_no', 'onsiterate', 'currency',
       'maxoccupancy', 'mealinclusiontype', 'var'],
      dtype='object')

# After applying Optimum band formula and simmilarity index

In [28]:
optimum_band=pd.read_csv('hotel_price_min_max - Formula.csv')

In [29]:
hotel.head()

,hotelcode,roomamenities,roomtype,ratedescription,hotelname,address,city,country,propertytype,starrating,latitude,longitude,url,guests_no
0,634876,"Air conditioning,Alarm clock,Carpeting,Closet,...",Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed",The Old Cider House,25 Castle Street,Nether Stowey,United Kingdom,Hotels,4,51.150921,-3.158470,https://www.agoda.com/en-gb/the-old-cider-hous...,2
1,7975,"Air conditioning,Clothes rack,Coffee/tea maker...",Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds",Apollo Hotel London,64-66 Queensborough Terrace,London,United Kingdom,Hotels,2,51.511234,-0.185117,https://www.agoda.com/en-gb/apollo-hotel-londo...,3
2,7975,"Air conditioning,Clothes rack,Coffee/tea maker...",Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds",Apollo Hotel London,64-66 Queensborough Terrace,London,United Kingdom,Hotels,2,51.511234,-0.185117,https://www.agoda.com/en-gb/apollo-hotel-londo...,3
3,7975,"Air conditioning,Clothes rack,Coffee/tea maker...",Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds",Apollo Hotel London,64-66 Queensborough Terrace,London,United Kingdom,Hotels,2,51.511234,-0.185117,https://www.agoda.com/en-gb/apollo-hotel-londo...,3
4,7975,"Air conditioning,Clothes rack,Heating,In-room ...",Deluxe Triple Room,"Shower, 1 single bed and 1 double bed",Apollo Hotel London,64-66 Queensborough Terrace,London,United Kingdom,Hotels,2,51.511234,-0.185117,https://www.agoda.com/en-gb/apollo-hotel-londo...,3


In [43]:
from math import sin, cos, sqrt, atan2, radians
import requests
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np

# Example secret value for API key
secret_value_0 = '4519d076b432f5'
url = "https://us1.locationiq.com/v1/search.php"

R = 6373.0  # Earth's Radius in kilometers
sw = stopwords.words('english')
lemm = WordNetLemmatizer()

def hybrid(address, city, number, rating, room_type, description, features):
    features = features.lower()
    room_type = room_type.lower()
    description = description.lower()

    # Tokenize and lemmatize the inputs
    features_tokens = word_tokenize(features)
    description_tokens = word_tokenize(description)

    f1_set = {w for w in features_tokens if not w in sw}
    d1_set = {w for w in description_tokens if not w in sw}

    f_set = set(lemm.lemmatize(se) for se in f1_set)
    d_set = set(lemm.lemmatize(se) for se in d1_set)

    data = {
        'key': secret_value_0,
        'q': address,
        'format': 'json'
    }
    response = requests.get(url, params=data)
    dist = []
    lat1, long1 = response.json()[0]['lat'], response.json()[0]['lon']
    lat1 = radians(float(lat1))
    long1 = radians(float(long1))

    hybridbase = hotel
    hybridbase['city'] = hybridbase['city'].str.lower()
    hybridbase = hybridbase[hybridbase['city'] == city.lower()]
    hybridbase.drop_duplicates(subset='hotelcode', inplace=True, keep='first')
    hybridbase = hybridbase[hybridbase['starrating'] >= rating]  # Filter by rating
    hybridbase = hybridbase.set_index(np.arange(hybridbase.shape[0]))

    for i in range(hybridbase.shape[0]):
        lat2 = radians(hybridbase['latitude'][i])
        long2 = radians(hybridbase['longitude'][i])
        dlon = long2 - long1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        dist.append(distance)

    hybridbase['distance'] = dist
    hybridbase = hybridbase[hybridbase['distance'] <= 5]
    hybridbase = hybridbase.set_index(np.arange(hybridbase.shape[0]))

    coss = []
    for i in range(hybridbase.shape[0]):
        temp_tokens = word_tokenize(hybridbase['roomamenities'][i])
        temp1_set = {w for w in temp_tokens if not w in sw}
        temp_set = set(lemm.lemmatize(se) for se in temp1_set)
        
        # Calculate similarity with features and description
        rvector = temp_set.intersection(f_set.union(d_set))
        similarity = len(rvector)

        # Check if room_type matches completely
        if room_type in hybridbase['roomtype'][i].lower():
            similarity += 1  # Increase similarity if room_type matches

        coss.append(similarity)

    hybridbase['similarity'] = coss
    return hybridbase.sort_values(by='similarity', ascending=False)


def price_band_based(address,city,number,rating,room_type,description,features):
    h=hybrid(address,city,number,rating,room_type,description,features)
    price_band=pd.merge(h,optimum_band,left_on=['hotelcode'],right_on=['hotelcode'],how='inner')
    price_band=pd.merge(price_band,hotel_cost,left_on=['hotelcode'],right_on=['hotelcode'],how='inner')
    del price_band['min']
    del price_band['max']
    del price_band['hotelcode']
    del price_band['Diff_Min']
    del price_band['Diff_Max']
    del price_band['currency']
    del price_band['propertytype']
    # del price_band['starrating']
    del price_band['latitude']
    del price_band['longitude']
    del price_band['guests_no']
    del price_band['var']
    # del price_band['distance']
    # del price_band['Score']
    # del price_band['maxoccupancy']
    del price_band['onsiterate']
    del price_band['ratedescription']
    price_band=price_band[price_band['Score']<=0.5]
    price_band=price_band[price_band['maxoccupancy']>=number]
    return price_band

In [44]:
price_band_based("London",'London',4,2,'double Room','Non-smoking','I need a Airconditioning and coffee maker.').head(5)

C:\Users\91949\AppData\Local\Temp\ipykernel_32304\575441772.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hybridbase.drop_duplicates(subset='hotelcode', inplace=True, keep='first')


,roomamenities,roomtype,hotelname,address,city,country,starrating,url,distance,similarity,Score,maxoccupancy,mealinclusiontype
4,"Air conditioning,Blackout curtains,Carpeting,C...",Single Room,Comfort Inn Victoria,18-24 Belgrave Road,london,United Kingdom,3,https://www.agoda.com/en-gb/comfort-inn-victor...,1.972495,2,0.48,4,Free breakfast for {4}
7,"Air conditioning,Alarm clock,Carpeting,Closet,...",Standard Twin Room,Princess Hotel,35-39 Argyle Street,london,United Kingdom,2,https://www.agoda.com/en-gb/princess-hotel/hot...,2.391054,2,0.16,4,No Complimentary
30,"Air conditioning,Alarm clock,Closet,Coffee/tea...",Double Room,Wedgewood Hotel,49-51 Leinster Square,london,United Kingdom,2,https://www.agoda.com/en-gb/wedgewood-hotel/ho...,4.519861,2,0.06,4,No Complimentary
47,"Air conditioning,Alarm clock,Closet,Clothes ra...",Executive Double Room,New Linden hotel,"58-60 Leinster Square,",london,United Kingdom,4,https://www.agoda.com/en-gb/new-linden-hotel/h...,4.612237,2,0.07,4,Free breakfast for {4}
75,"Air conditioning,Closet,Coffee/tea maker,Daily...",Single,Royal Eagle Hotel,26-30 Craven Road,london,United Kingdom,3,https://www.agoda.com/en-gb/royal-eagle-hotel/...,3.579570,1,0.44,4,No Complimentary
